In [1]:
import boto3
import pandas as pd
import s3fs
import re
import logging

from botocore.exceptions import ClientError


In [23]:
client = boto3.client('s3')

In [24]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('wsb-final-project-bucket')
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='aws-programmatic-access-test-object')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='kaggle_wsb_comments_12:6-2:6.csv')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='kaggle_wsb_posts_12:6-2:6.csv')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/b1a0304a-737f-4cc0-bff8-a7911ddf79ba/test-1.csv.gz')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/b1a0304a-737f-4cc0-bff8-a7911ddf79ba/test-Manifest.json')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/test-Manifest.json')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210201-20210301/305336a5-a905-4bf1-8b81-2f5d18953f59/test-1.csv.gz')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210201-20210301/305336a5-a905-4bf1-8b81-2f5d18953f59/test-Manifest.json')
s3.ObjectSummary

In [25]:
# comment_path = "s3://wsb-final-project-bucket/kaggle_wsb_comments_12:6-2:6.csv"
# post_path = "s3://wsb-final-project-bucket/kaggle_wsb_posts_12:6-2:6.csv"

In [26]:
post_path = "../Data/wallstreetbets_posts.csv"
comment_path = "../Data/wallstreetbets_comments.csv"

In [27]:
# comment_df = pd.read_csv(comment_path)
post_df = pd.read_csv(post_path)

In [28]:
# comment_df.head()

In [29]:
# post_df.columns.to_list()

In [30]:
post_df["title"]

0                                   Ruled by the Theta Gods
1                                        3k and I want more
2         Thank you fellow retards and autists! Big ston...
3         AZN fair value of $55, price of $50, vaccine c...
4                                  I wanna be less retarded
                                ...                        
707456    bro this is the most true shit ever and i can’...
707457    Took 5 years of losing on trading shares until...
707458                                                  💎 ✋
707459     Was it a dream??!😴 😴 GME opens on Monday up 100%
707460     Trading Plan 2021 - Need your constructive input
Name: title, Length: 707461, dtype: object

In [31]:
ticker_list = pd.read_csv("../Data/tickers.csv")

In [32]:
ticker_list

,"AA,0"
0,"AACG,0"
1,"AACQ,0"
2,"AACQU,0"
3,"AACQW,0"
4,"AAIC,0"
...,...
6929,"ZWRKW,0"
6930,"ZYME,0"
6931,"ZYNE,0"
6932,"ZYXI,0"


In [33]:
#Regex out commas and 0's in Tickers csv.
ticker_list = ticker_list.replace(',','', regex=True)
ticker_list = ticker_list.replace('0', '', regex=True)

In [34]:
ticker_list["Ticker"] = ticker_list["AA,0"]
ticker_list = ticker_list.drop(["AA,0"], 1)

In [35]:
ticker_list

,Ticker
0,AACG
1,AACQ
2,AACQU
3,AACQW
4,AAIC
...,...
6929,ZWRKW
6930,ZYME
6931,ZYNE
6932,ZYXI


In [36]:
ticker_list_test = ["GME", "AMC", "TSLA"]

In [37]:
# for row in post_df:
#     for ticker in ticker_list_test:
#         if post_df["title"].str.extract(rf"\s({ticker})\s", flags=re.IGNORECASE) == True:
#             ticker_name = ticker
#             post_df["ticker_name"] = ticker



    

In [38]:

for ticker in ticker_list_test:
    post_df['ticker_name'] = post_df['title'].str.extract(rf"\s({ticker})\s", expand=False).str.strip()
            

In [39]:
for ticker in ticker_list_test:
    post_df['ticker_name'] = post_df['title'].str.replace(rf"\s({ticker})\s", '').str.strip()

In [40]:
post_df["ticker_name"].describe()

count     707460
unique    619745
top          AMC
freq        1826
Name: ticker_name, dtype: object

In [41]:
post_df["ticker_name"] = None

In [42]:
index_title = post_df.columns.get_loc("title")
index_ticker_name = post_df.columns.get_loc("ticker_name")
index_ticker_list = ticker_list.columns.get_loc("Ticker")
print(index_title, index_ticker_name, index_ticker_list)

56 85 0


In [44]:
post_df['title']=post_df['title'].apply(str)

In [45]:
%%time
for row in range(0, 100):
    for ticker in ticker_list["Ticker"]:
        ticker_name = re.search(rf'\s({ticker})\s', post_df.iat[row, index_title], re.IGNORECASE)
        if ticker_name is not None:
            post_df.iat[row, index_ticker_name] = ticker_name.group()
            print(ticker, row)

CAR 9
BEPC 22
MTL 23
BOOM 31
CERT 32
EV 33
FORD 33
CD 35
PS 35
BFT 39
GOOD 45
PUMP 51
PUMP 53
GOOD 61
GME 64
LI 88
NIO 88
XPEV 88
NIO 94
WKHS 96
CPU times: user 1min 18s, sys: 829 ms, total: 1min 19s
Wall time: 1min 19s


In [55]:
%%time
count = 0
for row in post_df.itertuples():
    for ticker in ticker_list["Ticker"]:
        individual_ticker_name = re.search(rf'\s({ticker})\s', row.title, re.IGNORECASE)
        if individual_ticker_name is not None:
            post_df.iloc[row.index][index_ticker_name] = individual_ticker_name
            

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [54]:
post_df.isna().sum()

all_awardings                         0
allow_live_comments                   0
author                                0
author_flair_background_color    706340
author_flair_css_class           707385
                                  ...  
distinguished                    707427
collections                      707433
crosspost_parent                 707460
crosspost_parent_list            707460
ticker_name                      707445
Length: 86, dtype: int64

In [49]:
print(post_df.iloc[9][index_ticker_name])

 car 
